In [1]:
import cupy as cp
import numpy as np

ModuleNotFoundError: No module named 'cupy'

In [ ]:
# array on host memory

x_host = np.array([1,2,3])
type(x_host)

In [ ]:
x_device=cp.array([1,2,3])
type(x_device)

In [ ]:
%%timeit
np.linalg.norm(x_host)

In [ ]:
%%timeit
cp.linalg.norm(x_device)

In [ ]:
with cp.cuda.Device(0):
    x_on_device=cp.array([1,2,3,4,5])

In [ ]:
# Create numpy array on host
x_host = np.random.randint(0,255,(2000,2000))

In [ ]:
# Transfer numpy array from host to device
x_device = cp.asarray(x_host)

In [ ]:
# Transfer supy array from device to host 
x_host_1= x_device.get()

In [ ]:
# compute FFT
from scipy.fft import fftn


In [ ]:
%%timeit
fftn(x_host)

In [ ]:
# CUDA implememtnions
import cupyx

In [ ]:
%%timeit
cupyx.scipy.fft.fftn(x_device)

# Gauss Blur image

In [6]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


def generate_gaussian_filter(sigma: int | float,filter_shape: list | tuple | None):
    # 'sigma' is the standard deviation of the gaussian distribution

    m, n = filter_shape
    m_half = m // 2
    n_half = n // 2

    # initializing the filter
    gaussian_filter = np.zeros((m, n), np.float32)

    # generating the filter
    for y in range(-m_half, m_half):
        for x in range(-n_half, n_half):
            normal = 1 / (2.0 * np.pi * sigma**2.0)
            exp_term = np.exp(-(x**2.0 + y**2.0) / (2.0 * sigma**2.0))
            gaussian_filter[y+m_half, x+n_half] = normal * exp_term

    return gaussian_filter  # Add this line



"""
Created on Wed Jan  4 12:06:28 2023
@author: rohit krishna
@mail   : dev.rohitnp@gmail.com
"""

import numpy as np


def convolution(image: np.ndarray, kernel: list | tuple) -> np.ndarray:
    '''
    It is a "valid" Convolution algorithm implementaion.
    ### Example
    >>> import numpy as np
    >>> from PIL import Image
    >>>
    >>> kernel = np.array(
    >>>     [[-1, 0, 1],
    >>>     [-2, 0, 2],
    >>>     [-1, 0, 1]], np.float32
    >>> )
    >>> img = np.array(Image.open('./lenna.png'))
    >>> res = convolution(img, Kx)
    '''

    if len(image.shape) == 3:
        m_i, n_i, c_i = image.shape

    # if the image is gray then we won't be having an extra channel so handling it
    elif len(image.shape) == 2:
        image = image[..., np.newaxis]
        m_i, n_i, c_i = image.shape
    else:
        raise Exception('Shape of image not supported')

    m_k, n_k = kernel.shape

    y_strides = m_i - m_k + 1  # possible number of strides in y direction
    x_strides = n_i - n_k + 1  # possible number of strides in x direction

    img = image.copy()
    output_shape = (m_i-m_k+1, n_i-n_k+1, c_i)
    output = np.zeros(output_shape, dtype=np.float32)

    count = 0  # taking count of the convolution operation being happening

    output_tmp = output.reshape(
        (output_shape[0]*output_shape[1], output_shape[2])
    )

    for i in range(y_strides):
        for j in range(x_strides):
            for c in range(c_i): # looping over the all channels
                sub_matrix = img[i:i+m_k, j:j+n_k, c]

                output_tmp[count, c] = np.sum(sub_matrix * kernel)

            count += 1

    output = output_tmp.reshape(output_shape)

    return output

# Generate Gaussian filter
sigma = 1.0  # Standard deviation
filter_shape = (5, 5)  # Filter size
gaussian_filter = generate_gaussian_filter(sigma, filter_shape)

# Load image
img = np.array(Image.open('./bird.png'))

# Apply convolution with Gaussian filter
res = convolution(img, gaussian_filter)

# Display the result
plt.imshow(res.astype(np.uint8))
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/High-Performance-Computing-practice/Code_3/bird.png'

# cupy

In [8]:
import numpy as np
import cupy as cp
from cupyx.scipy.signal import convolve2d as convolve2d_gpu
from PIL import Image
import matplotlib.pyplot as plt

def generate_gaussian_filter(sigma: int | float, filter_shape: list | tuple | None):
    m, n = filter_shape
    m_half = m // 2
    n_half = n // 2

    gaussian_filter = np.zeros((m, n), np.float32)

    for y in range(-m_half, m_half):
        for x in range(-n_half, n_half):
            normal = 1 / (2.0 * np.pi * sigma**2.0)
            exp_term = np.exp(-(x**2.0 + y**2.0) / (2.0 * sigma**2.0))
            gaussian_filter[y+m_half, x+n_half] = normal * exp_term

    return gaussian_filter

def convolution(image: np.ndarray, kernel: list | tuple) -> np.ndarray:
    if len(image.shape) == 3:
        m_i, n_i, c_i = image.shape
    elif len(image.shape) == 2:
        image = image[..., np.newaxis]
        m_i, n_i, c_i = image.shape
    else:
        raise Exception('Shape of image not supported')

    output = np.zeros_like(image, dtype=np.float32)

    for c in range(c_i):
        output[..., c] = convolve2d_gpu(cp.asarray(image[..., c]), cp.asarray(kernel), mode='same').get()

    return output

sigma = 1.0
filter_shape = (5, 5)
gaussian_filter = generate_gaussian_filter(sigma, filter_shape)

img = np.array(Image.open('./bird.png'))

res = convolution(img, gaussian_filter)

plt.imshow(res.astype(np.uint8))
plt.show()

ModuleNotFoundError: No module named 'cupy'

# Mean 

In [ ]:
import numpy as np
import cupy as cp
from cupyx.scipy.signal import convolve2d as convolve2d_gpu
from PIL import Image
import matplotlib.pyplot as plt

def generate_mean_filter(filter_shape: list | tuple | None):
    m, n = filter_shape
    return np.ones((m, n), np.float32) / (m * n)

def convolution(image: np.ndarray, kernel: list | tuple) -> np.ndarray:
    if len(image.shape) == 3:
        m_i, n_i, c_i = image.shape
    elif len(image.shape) == 2:
        image = image[..., np.newaxis]
        m_i, n_i, c_i = image.shape
    else:
        raise Exception('Shape of image not supported')

    output = np.zeros_like(image, dtype=np.float32)

    for c in range(c_i):
        output[..., c] = convolve2d_gpu(cp.asarray(image[..., c]), cp.asarray(kernel), mode='same').get()

    return output                     

sigma = 1.0
filter_shape = (5, 5)
gaussian_filter = generate_gaussian_filter(sigma, filter_shape)

img = np.array(Image.open('./bird.png'))

res = convolution(img, gaussian_filter)

plt.imshow(res.astype(np.uint8))
plt.show()

# Grayscale